In [9]:
import time
from datetime import datetime, timedelta
from redditClient import redditClient
from prawcore.exceptions import TooManyRequests, RequestException
import logging
import json
from tqdm import tqdm  # Import tqdm for progress monitoring

# Configure logging to see detailed output
logging.basicConfig(level=logging.INFO)

def get_rate_limit_info(reddit):
    """
    Fetch rate limit information from the last API response headers.
    Handles missing 'reset' or 'remaining' keys.
    """
    rate_limit_used = reddit.auth.limits.get('used', 0)
    rate_limit_remaining = reddit.auth.limits.get('remaining', 100)  # Default to 100 if not present
    rate_limit_reset = reddit.auth.limits.get('reset', 60)  # Default to 60 seconds if not present

    return rate_limit_used, rate_limit_remaining, rate_limit_reset

def fetch_replies(comment, max_replies=5, max_nested_replies=3):
    """
    Recursively fetch replies to a comment, with limits on nested replies.
    
    @param comment: PRAW comment object
    @param max_replies: Maximum number of direct replies to fetch
    @param max_nested_replies: Maximum number of nested replies to fetch
    @return: List of reply dictionaries
    """
    replies_data = []
    try:
        comment.replies.replace_more(limit=None)  # Load all replies for the comment
        replies = comment.replies.list()[:max_replies]  # Limit the number of direct replies

        for reply in replies:
            reply_data = {
                'comment_id': reply.id,
                'body': reply.body,
                'author': reply.author.name if reply.author else '[deleted]',
                'score': reply.score,
                'created_utc': reply.created_utc,
                'parent_id': reply.parent_id,
                'replies': []
            }
            # Recursively fetch nested replies, with a limit on how deep to go
            if max_nested_replies > 0:
                reply_data['replies'] = fetch_replies(reply, max_replies=max_replies, max_nested_replies=max_nested_replies - 1)
            replies_data.append(reply_data)
    except Exception as e:
        logging.error(f"Error fetching replies: {e}")
    
    return replies_data

def scrape_top_daily_posts_comments(max_posts=20, max_comments=10, max_replies=5, max_nested_replies=3):
    """
    Scrape the top posts for each day from r/politics over the last 30 days, including Redditor usernames and comments.
    Includes nested replies with limits on the number of replies and nested replies.
    
    @param max_posts: Maximum number of posts to scrape per day
    @param max_comments: Maximum number of comments to scrape per post
    @param max_replies: Maximum number of replies per top-level comment
    @param max_nested_replies: Maximum number of nested replies to fetch recursively
    @return: List of dictionaries containing post and comment details
    """
    # Initialize Reddit client
    reddit = redditClient()

    # Access the 'politics' subreddit
    subreddit = reddit.subreddit('politics')

    # Calculate the timestamp for 30 days ago
    today = datetime.utcnow()
    thirty_days_ago = today - timedelta(days=30)

    # List to store the scraped data
    scraped_data = []

    # Loop over each day in the last 30 days
    for day in tqdm(range(30), desc="Processing last 30 days"):
        day_end = today - timedelta(days=day)
        day_start = day_end - timedelta(days=1)

        logging.info(f"Fetching top {max_posts} posts from {day_start.strftime('%Y-%m-%d')} to {day_end.strftime('%Y-%m-%d')}")

        try:
            # Fetch top posts for the day
            top_posts = list(subreddit.top(limit=max_posts, time_filter='day'))

            # Create a progress bar for the posts processing
            for post in tqdm(top_posts, desc=f"Processing posts for {day_start.strftime('%Y-%m-%d')}", leave=False):
                # Get the author's username or 'deleted' if the author is None
                post_author = post.author.name if post.author else '[deleted]'

                post_data = {
                    'title': post.title,
                    'id': post.id,
                    'author': post_author,
                    'score': post.score,
                    'url': post.url,
                    'num_comments': post.num_comments,
                    'created_utc': post.created_utc,
                    'upvote_ratio': post.upvote_ratio,
                    'comments': []
                }

                # Fetch comments for each post
                time.sleep(1)  # Add a 1 second delay to reduce chances of hitting rate limits
                try:
                    post.comments.replace_more(limit=None)  # To ensure all comments are loaded
                except TooManyRequests:
                    logging.warning("Too many requests when loading comments. Sleeping for 5 seconds...")
                    time.sleep(5)  # Sleep for a few seconds before retrying
                    continue

                # Limit top-level comments
                top_comments = post.comments.list()[:max_comments]
                for comment in top_comments:
                    comment_author = comment.author.name if comment.author else '[deleted]'
                    comment_data = {
                        'comment_id': comment.id,
                        'body': comment.body,
                        'author': comment_author,
                        'score': comment.score,
                        'created_utc': comment.created_utc,
                        'parent_id': comment.parent_id,
                        'replies': fetch_replies(comment, max_replies=max_replies, max_nested_replies=max_nested_replies)  # Fetch replies recursively
                    }
                    post_data['comments'].append(comment_data)

                scraped_data.append(post_data)

            # Monitor rate limit headers after fetching posts
            used, remaining, reset_time = get_rate_limit_info(reddit)
            logging.info(f"Rate Limit Info: Used: {used}, Remaining: {remaining}, Reset in: {reset_time} seconds")

            # If the remaining rate limit is very low, pause until the reset time
            if remaining < 5:
                logging.info(f"Approaching rate limit, sleeping for {reset_time} seconds...")
                time.sleep(reset_time)  # Sleep until rate limit reset

        except RequestException as e:
            logging.error(f"Request failed: {e}. Retrying...")
            retries = 0
            while retries < 5:
                retries += 1
                logging.info(f"Retrying... attempt {retries}/5")
                time.sleep(5 * retries)  # Exponential backoff
                try:
                    top_posts = subreddit.top(limit=max_posts, time_filter='day')
                    break
                except RequestException:
                    if retries == 5:
                        logging.error("Max retries exceeded. Aborting.")
                        raise
                    else:
                        continue

        except TooManyRequests as e:
            # Handle Reddit API rate limiting by backing off and retrying
            logging.warning("Hit the Reddit rate limit. Sleeping for 10 seconds...")
            time.sleep(10)  # Sleep for 10 seconds and then retry
            continue

        # Add a small delay between requests to prevent breaching rate limits
        time.sleep(1)  # 1 second delay between each API call to stay safe

    return scraped_data

# Example usage
if __name__ == "__main__":
    # Scrape the data
    data = scrape_top_daily_posts_comments(max_posts=20, max_comments=10, max_replies=5, max_nested_replies=3)

    # Print the scraped data length to verify
    logging.info(f"Total posts scraped: {len(data)}")

    # Save the data to a JSON file for further analysis
    with open("r_politics_top_daily_posts_and_comments.json", "w") as outfile:
        json.dump(data, outfile, indent=4)

Processing last 30 days:   0%|                           | 0/30 [00:00<?, ?it/s]INFO:root:Fetching top 20 posts from 2024-10-17 to 2024-10-18

Processing posts for 2024-10-17: 100%|██████████| 20/20 [09:52<00:00,  5.22s/it]
                                                                                INFO:root:Rate Limit Info: Used: 588, Remaining: 412.0, Reset in: 60 seconds
Processing last 30 days:   3%|▌               | 1/30 [09:55<4:47:55, 595.70s/it]INFO:root:Fetching top 20 posts from 2024-10-16 to 2024-10-17

Processing posts for 2024-10-16:   5%|▌          | 1/20 [01:39<31:27, 99.34s/it]WARNING:root:Too many requests when loading comments. Sleeping for 5 seconds...

Processing posts for 2024-10-16: 100%|██████████| 20/20 [09:52<00:00,  5.46s/it]
                                                                                INFO:root:Rate Limit Info: Used: 575, Remaining: 425.0, Reset in: 60 seconds
Processing last 30 days:   7%|█               | 2/30 [19:49<4:37:34, 594.81s/

Processing posts for 2024-10-14: 100%|██████████| 20/20 [11:11<00:00,  5.40s/it]
                                                                                INFO:root:Rate Limit Info: Used: 698, Remaining: 302.0, Reset in: 60 seconds
Processing last 30 days:  13%|██▏             | 4/30 [41:01<4:31:35, 626.73s/it]INFO:root:Fetching top 20 posts from 2024-10-13 to 2024-10-14

Processing posts for 2024-10-13: 100%|██████████| 20/20 [11:23<00:00,  5.36s/it]
                                                                                INFO:root:Rate Limit Info: Used: 839, Remaining: 161.0, Reset in: 60 seconds
Processing last 30 days:  17%|██▋             | 5/30 [52:26<4:29:55, 647.80s/it]INFO:root:Fetching top 20 posts from 2024-10-12 to 2024-10-13

Processing posts for 2024-10-12: 100%|██████████| 20/20 [11:58<00:00,  5.44s/it]
                                                                                INFO:root:Rate Limit Info: Used: 45, Remaining: 955.0, Reset in: 60 seconds
P

Processing posts for 2024-10-10: 100%|██████████| 20/20 [12:16<00:00,  5.72s/it]
                                                                                INFO:root:Rate Limit Info: Used: 496, Remaining: 504.0, Reset in: 60 seconds
Processing last 30 days:  27%|███▋          | 8/30 [1:28:57<4:19:09, 706.78s/it]INFO:root:Fetching top 20 posts from 2024-10-09 to 2024-10-10

Processing posts for 2024-10-09: 100%|██████████| 20/20 [13:15<00:00,  5.96s/it]
                                                                                INFO:root:Rate Limit Info: Used: 825, Remaining: 175.0, Reset in: 60 seconds
Processing last 30 days:  30%|████▏         | 9/30 [1:42:15<4:17:19, 735.20s/it]INFO:root:Fetching top 20 posts from 2024-10-08 to 2024-10-09

Processing posts for 2024-10-08:   0%|                   | 0/20 [00:00<?, ?it/s]WARNING:root:Too many requests when loading comments. Sleeping for 5 seconds...

Processing posts for 2024-10-08: 100%|██████████| 20/20 [10:11<00:00,  5.96s/

Processing posts for 2024-10-06: 100%|██████████| 20/20 [12:20<00:00,  5.47s/it]
                                                                                INFO:root:Rate Limit Info: Used: 303, Remaining: 697.0, Reset in: 60 seconds
Processing last 30 days:  40%|█████▏       | 12/30 [2:17:11<3:36:04, 720.24s/it]INFO:root:Fetching top 20 posts from 2024-10-05 to 2024-10-06

Processing posts for 2024-10-05:  10%|█         | 2/20 [06:00<51:00, 170.03s/it]WARNING:root:Too many requests when loading comments. Sleeping for 5 seconds...

Processing posts for 2024-10-05: 100%|██████████| 20/20 [11:42<00:00,  5.94s/it]
                                                                                INFO:root:Rate Limit Info: Used: 477, Remaining: 523.0, Reset in: 60 seconds
Processing last 30 days:  43%|█████▋       | 13/30 [2:28:55<3:22:40, 715.35s/it]INFO:root:Fetching top 20 posts from 2024-10-04 to 2024-10-05

Processing posts for 2024-10-04:   5%|▍       | 1/20 [04:03<1:16:58, 243.09s/

Processing posts for 2024-10-02: 100%|██████████| 20/20 [13:23<00:00,  6.44s/it]
                                                                                INFO:root:Rate Limit Info: Used: 280, Remaining: 720.0, Reset in: 60 seconds
Processing last 30 days:  53%|██████▉      | 16/30 [3:06:54<2:55:45, 753.26s/it]INFO:root:Fetching top 20 posts from 2024-10-01 to 2024-10-02

Processing posts for 2024-10-01: 100%|██████████| 20/20 [13:13<00:00,  6.55s/it]
                                                                                INFO:root:Rate Limit Info: Used: 613, Remaining: 387.0, Reset in: 60 seconds
Processing last 30 days:  57%|███████▎     | 17/30 [3:20:09<2:45:57, 765.95s/it]INFO:root:Fetching top 20 posts from 2024-09-30 to 2024-10-01

Processing posts for 2024-09-30:   0%|                   | 0/20 [00:00<?, ?it/s]WARNING:root:Too many requests when loading comments. Sleeping for 5 seconds...

Processing posts for 2024-09-30: 100%|██████████| 20/20 [12:32<00:00,  6.79s/

Processing posts for 2024-09-28: 100%|██████████| 20/20 [13:13<00:00,  7.24s/it]
                                                                                INFO:root:Rate Limit Info: Used: 545, Remaining: 455.0, Reset in: 60 seconds
Processing last 30 days:  67%|████████▋    | 20/30 [3:59:24<2:10:11, 781.17s/it]INFO:root:Fetching top 20 posts from 2024-09-27 to 2024-09-28

Processing posts for 2024-09-27:   0%|                   | 0/20 [00:00<?, ?it/s]WARNING:root:Too many requests when loading comments. Sleeping for 5 seconds...

Processing posts for 2024-09-27: 100%|██████████| 20/20 [11:30<00:00,  4.87s/it]
                                                                                INFO:root:Rate Limit Info: Used: 670, Remaining: 330.0, Reset in: 60 seconds
Processing last 30 days:  70%|█████████    | 21/30 [4:10:56<1:53:09, 754.43s/it]INFO:root:Fetching top 20 posts from 2024-09-26 to 2024-09-27

Processing posts for 2024-09-26: 100%|██████████| 20/20 [11:31<00:00,  4.69s/

Processing posts for 2024-09-24: 100%|██████████| 20/20 [11:44<00:00,  5.28s/it]
                                                                                INFO:root:Rate Limit Info: Used: 198, Remaining: 802.0, Reset in: 60 seconds
Processing last 30 days:  80%|██████████▍  | 24/30 [4:46:15<1:12:22, 723.70s/it]INFO:root:Fetching top 20 posts from 2024-09-23 to 2024-09-24

Processing posts for 2024-09-23:  10%|█         | 2/20 [06:25<54:01, 180.11s/it]WARNING:root:Too many requests when loading comments. Sleeping for 5 seconds...

Processing posts for 2024-09-23: 100%|██████████| 20/20 [11:23<00:00,  6.75s/it]
                                                                                INFO:root:Rate Limit Info: Used: 348, Remaining: 652.0, Reset in: 60 seconds
Processing last 30 days:  83%|████████████▌  | 25/30 [4:57:40<59:21, 712.20s/it]INFO:root:Fetching top 20 posts from 2024-09-22 to 2024-09-23

Processing posts for 2024-09-22:   5%|▍       | 1/20 [04:24<1:23:40, 264.24s/


Processing posts for 2024-09-20: 100%|██████████| 20/20 [10:03<00:00, 11.66s/it]
                                                                                INFO:root:Rate Limit Info: Used: 841, Remaining: 159.0, Reset in: 60 seconds
Processing last 30 days:  93%|██████████████ | 28/30 [5:32:17<22:57, 688.66s/it]INFO:root:Fetching top 20 posts from 2024-09-19 to 2024-09-20

Processing posts for 2024-09-19:  45%|████▉      | 9/20 [11:05<06:25, 35.06s/it]WARNING:root:Too many requests when loading comments. Sleeping for 5 seconds...

Processing posts for 2024-09-19: 100%|██████████| 20/20 [13:31<00:00, 12.51s/it]
                                                                                INFO:root:Rate Limit Info: Used: 197, Remaining: 803.0, Reset in: 60 seconds
Processing last 30 days:  97%|██████████████▌| 29/30 [5:45:49<12:05, 725.92s/it]INFO:root:Fetching top 20 posts from 2024-09-18 to 2024-09-19

Processing posts for 2024-09-18:  10%|█         | 2/20 [06:52<57:26, 191.46s